In [1]:
from prody import *
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


pdb_files = sorted(glob.glob("*1000.pdb"))

if not pdb_files:
    print("No *1000.pdb files found in the current directory.")

for pdb_file in pdb_files:
    stem = os.path.splitext(os.path.basename(pdb_file))[0]   
    dcd_file = f"{stem}.dcd"                                 

    if not os.path.isfile(dcd_file):
        print(f"Skipping {stem}: missing {dcd_file}")
        continue

    print(f"\nProcessing: {pdb_file} + {dcd_file}")

    # --- read structure and trajectory ---
    pdb = parsePDB(pdb_file)
    dcd = parseDCD(dcd_file)

    # --- selections (same as your code) ---
    n_terminal = pdb.select("resnum 5 to 68 and calpha")
    linker     = pdb.select("resnum 69 to 91 and calpha")
    c_terminal = pdb.select("resnum 92 to 148 and calpha")
    overall    = pdb.select("calpha")

    print("N-terminal atoms:", n_terminal)
    print("Linker atoms:", linker)
    print("C-terminal atoms:", c_terminal)

    regions = {
        "N-terminal": n_terminal,
        "Linker": linker,
        "C-terminal": c_terminal,
        "All": overall
    }

    rmsd_results = {}

    # --- compute RMSD per region ---
    for region_name, region in regions.items():
        if region is not None:
            dcd.setCoords(pdb)
            dcd.setAtoms(region)
            dcd.superpose()
            rmsd = dcd.getRMSDs()
            rmsd_results[region_name] = rmsd
            print(f"{region_name} RMSD values:", rmsd)
        else:
            print(f"No atoms selected for {region_name}")

    if not rmsd_results:
        print(f"Skipping plot for {stem}: no RMSD results.")
        continue

    # --- plot and save ---
    plt.figure(figsize=(10, 4))

    colors = ["forestgreen", "hotpink", "teal", "black"]

    max_rmsd = max([max(r) for r in rmsd_results.values()])
    min_rmsd = min([min(r) for r in rmsd_results.values()])

    for i, (region_name, rmsd) in enumerate(rmsd_results.items()):
        plt.plot(range(len(rmsd)), rmsd, label=f"{region_name} RMSD", color=colors[i % len(colors)])

    # Keep your 0..1000 style if you want; otherwise this adapts automatically
    if len(next(iter(rmsd_results.values()))) > 1000:
        plt.xlim(0, 1000)
    else:
        plt.xlim(0, len(next(iter(rmsd_results.values()))) - 1)

    plt.ylim(min_rmsd, max(15, max_rmsd + 0.5))

    plt.xlabel("Time(ns)", fontsize=16)
    plt.ylabel("RMSD(Å)", fontsize=16)
    plt.legend(loc="upper left")

    out_png = f"{stem}_rmsd.png"
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"Saved: {out_png}")


@> 2184 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> DCD file contains 1001 coordinate sets for 2184 atoms.
@> DCD file was parsed in 0.05 seconds.
@> 25.10 MB parsed at input rate 461.02 MB/s.
@> 1001 coordinate sets parsed at input rate 18389 frame/s.
@> Superposing  [ 10%] 2s


Processing: apo_l_1000.pdb + apo_l_1000.dcd
N-terminal atoms: Selection 'resnum 5 to 68 and calpha'
Linker atoms: Selection 'resnum 69 to 91 and calpha'
C-terminal atoms: Selection 'resnum 92 to 148 and calpha'


@> Superposition completed in 1.04 seconds.
@> Superposing  [ 20%] 1s

N-terminal RMSD values: [5.71294119e-06 5.71294119e-06 2.63682692e+00 ... 6.14380891e+00
 5.88874945e+00 6.01198307e+00]


@> Superposition completed in 0.94 seconds.
@> Superposing  [ 20%] 1s

Linker RMSD values: [3.12200846e-06 3.12200846e-06 7.95864999e-01 ... 1.13286085e+00
 1.07209352e+00 1.15007900e+00]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

@> Superposition completed in 0.84 seconds.


All RMSD values: [7.25088259e-06 7.25088259e-06 4.19384628e+00 ... 6.52540105e+00
 6.27602076e+00 6.29962351e+00]
Saved: apo_l_1000_rmsd.png
